In [ ]:
from matplotlib import pyplot as plt
from matplotlib import ticker as mticker
from matplotlib import dates as mdates
from matplotlib.cm import get_cmap
from pathlib import Path
import numpy as np
import json
from collections import namedtuple
from dateutil.parser import isoparse
from collections import defaultdict
from datetime import datetime

DATA_DIR = Path("~/OneDrive/Documents/pingtest").expanduser()
TIMEOUT = 4294967295
MAX_IPS = 3

In [ ]:
Record = namedtuple("Record", ["start_time", "duration", "interval", "targets"])
Target = namedtuple("Target", ["host_name", "ip", "pings"])
Dataset = namedtuple("Dataset", ["host_name", "ips", "pings"])

def parseTarget(target) -> Target:
    target = Target(**target)
    m = map(lambda x: (x["started_at"], x["rtt"]), target.pings)
    target = target._replace(pings = np.array([r for r in m if r[1] != TIMEOUT]))
    return target

def loadData(path: Path) -> Record:
    data = json.loads(path.read_text())
    rec = Record(**data)
    rec = rec._replace(
        targets = list(map(parseTarget, rec.targets)),
        start_time = np.datetime64(isoparse(rec.start_time).replace(tzinfo=None), "ms"))
    return rec

def transformPings(startTime: np.datetime64, pings: np.ndarray):
    time = startTime + pings[:,0].astype("timedelta64[ms]")
    rtt = pings[:,1].astype("timedelta64[ms]")
    return time, rtt

def joinDatasets(records: list[Record]) -> list[Dataset]:
    sets = defaultdict(lambda: Dataset("", set(), (np.array([], dtype="datetime64[ms]"), np.array([], dtype="timedelta64[ms]"))))
    for record in records:
        for target in record.targets:
            time, rtt = transformPings(record.start_time, target.pings)
            key = target.host_name if len(target.host_name) > 0 else target.ip
            d = sets[key]
            d.ips.add(target.ip)
            d = Dataset(target.host_name, d.ips, (np.r_[d.pings[0], time], np.r_[d.pings[1], rtt]))
            sets[key] = d

    for k, ds in sets.items():
        sets[k] = ds._replace(ips = list(ds.ips))

    return list(sets.values())

def takeRecent(datasets: list[Dataset], timeSpan: np.timedelta64 = np.timedelta64(24, 'h')):
    def update(ds):
        time, rtt = ds.pings
        threshold = time[-1] - timeSpan
        indices = time > threshold
        return ds._replace(pings = (time[indices], rtt[indices]))
    return list(map(update, datasets))

def removeDate(datasets: list[Dataset]):
    def update(ds):
        time, rtt = ds.pings
        time = time + np.timedelta64(30, 'm')
        time = time - time.astype('datetime64[D]')
        return ds._replace(pings = (time, rtt))
    return list(map(update, datasets))

def groupByTimeOfDay(datasets: list[Dataset]):
    def update(ds):
        time = time.astype('timedelta64[h]')
        a = [(t, rtt[time == t]) for t in np.unique(time)]
        return ds._replace(pings = a)
    return list(map(update, removeDate(datasets)))

def histogram(datasets, bins):
    def hist(rtt):
        h, _ = np.histogram(rtt, bins)
        return h.astype('float64') / np.sum(h)
    def update(ds):
        h = [(t, hist(rtt)) for t, rtt in ds.pings]
        return ds._replace(pings = h)
    return list(map(update, datasets))

In [ ]:
datasets = [loadData(x) for x in DATA_DIR.iterdir() if x.is_file()]
datasets = joinDatasets(datasets)

In [ ]:
recent = takeRecent(datasets)

fig, axs = plt.subplots(len(recent), 1, sharex=True)
fig.set_dpi(150)
fig.set_size_inches(24, 6 * len(recent))
fig.patch.set_facecolor("white")

for i, (target, ax) in enumerate(zip(recent, axs)):
    time = target.pings[0].astype(datetime)
    rtt = target.pings[1]

    ips = str.join(', ', target.ips[:MAX_IPS])
    if len(target.ips) > MAX_IPS:
        ips += ", ..."
    title = f"{target.host_name} ({ips})" if len(target.host_name) > 0 else ips
    ax.set_title(title)
    ax.grid(axis="y")
    ax.tick_params(labelbottom=True)
    # ax.set_xlabel("Time")
    # ax.set_ylabel("Ping")
    ax.set_ylim((1, 1000))
    ax.set_yscale("symlog", linthresh=100)
    ax.set_yticks([0, 20, 40, 60, 80, 100, 200, 400, 800])
    ax.yaxis.set_major_formatter(mticker.FormatStrFormatter("%i ms"))
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(ax.xaxis.get_major_locator()))

    ax.axhline(100, c='red', linewidth=1, alpha=0.5)
    ax.axhline(60, c='orange', linewidth=1, alpha=0.5)

    above100 = np.count_nonzero(rtt > np.timedelta64(100, "ms")) / len(rtt)
    above60 = np.count_nonzero(rtt > np.timedelta64(60, "ms")) / len(rtt)
    stats = f">100ms: {above100:.2%}\n>60ms: {above60:.2%}"
    ax.text(0.08, 0.85, stats, transform=ax.transAxes, horizontalalignment="right", bbox=dict(facecolor="white", alpha=0.7))

    ax.plot(time, rtt, ".", markersize=1, c=f"C{i}", alpha=0.5)

plt.show()

In [ ]:
bins = [0, 30, 50, 100, 1000]
hourGroups = groupByTimeOfDay(datasets)
histGroups = histogram(hourGroups, bins)
pingCount = sum([len(ds.pings[0]) for ds in datasets]) // len(datasets)
startDate = min([min(ds.pings[0]) for ds in datasets])
startDate = str(startDate)[:10]
endDate = max([max(ds.pings[0]) for ds in datasets])
endDate = str(endDate)[:10]

cmap = get_cmap('turbo')
colors = [ "#1ec943", "#f3e438", "#f7870f", "#d11010", "#760000" ]

fig, axs = plt.subplots(len(histGroups), 1, sharex=True)
fig.set_dpi(150)
fig.set_size_inches(24, 6 * len(histGroups))
fig.patch.set_facecolor("white")
fig.suptitle(f"Hourly ping histogram over {pingCount:,} pings between {startDate} and {endDate}", fontsize=18)

for i, (target, ax) in enumerate(zip(histGroups, axs)):
    times, hists = zip(*target.pings)
    hists = np.array(hists) * 100
    cumhists = np.cumsum(hists, 1)
    times = np.array(times).astype('int32')

    ips = str.join(', ', target.ips[:MAX_IPS])
    if len(target.ips) > MAX_IPS:
        ips += ", ..."
    title = f"{target.host_name} ({ips})" if len(target.host_name) > 0 else ips

    ax.set_title(title)
    ax.tick_params(labelbottom=True)
    ax.yaxis.set_major_formatter(mticker.FormatStrFormatter("%i %%"))
    ax.xaxis.set_major_formatter(mticker.FormatStrFormatter("%i h"))
    ax.set_xticks(range(0, 24))

    for i in reversed(range(cumhists.shape[1])):
        bot = cumhists[:, i - 1] if i > 0 else 0
        b = ax.bar(x=times, height=hists[:,i], bottom=bot, color=colors[i], label=f'<{bins[i + 1]} ms')
        ax.bar_label(b, label_type='center', fmt='%i %%')

    ax.legend(title="Ping")

plt.show()

In [ ]:
hourGroups = removeDate(datasets)

fig, axs = plt.subplots(len(hourGroups), 1, sharex=True)
fig.set_dpi(150)
fig.set_size_inches(24, 6 * len(hourGroups))
fig.patch.set_facecolor("white")

for i, (target, ax) in enumerate(zip(hourGroups, axs)):
    times, rtt = target.pings
    times = times.astype('float64') / 1000 / 60 / 60

    ips = str.join(', ', target.ips[:MAX_IPS])
    if len(target.ips) > MAX_IPS:
        ips += ", ..."
    title = f"{target.host_name} ({ips})" if len(target.host_name) > 0 else ips
    ax.set_title(title)
    ax.grid(axis="y")
    ax.tick_params(labelbottom=True)
    # ax.set_xlabel("Time")
    # ax.set_ylabel("Ping")
    ax.set_ylim((1, 1000))
    ax.set_yscale("symlog", linthresh=100)
    ax.set_yticks([0, 20, 40, 60, 80, 100, 200, 400, 800])
    ax.yaxis.set_major_formatter(mticker.FormatStrFormatter("%i ms"))
    ax.xaxis.set_major_formatter(mticker.FormatStrFormatter("%i h"))
    ax.set_xticks(range(0, 24))

    ax.axhline(100, c='red', linewidth=1, alpha=0.5)
    ax.axhline(60, c='orange', linewidth=1, alpha=0.5)

    ax.plot(times, rtt, ".", markersize=1, c=f"C{i}", alpha=0.1)

plt.show()